In [304]:
import serial
import time
import csv
from bs4 import BeautifulSoup
import subprocess
import atexit

ser = serial.Serial('/dev/cu.usbmodem14301')
ser.flushInput()

bar_pompw = 'arduino_local_csv/barcode_item_pomp_water.html'
bar_pompa = 'arduino_local_csv/barcode_item_pomp_afvalvat.html'
bar_pomps1 = 'arduino_local_csv/barcode_item_pomp_aardbei.html'
bar_pomps2 = 'arduino_local_csv/barcode_item_pomp_appel.html'
bar_pomps3 = 'arduino_local_csv/barcode_item_pomp_sinaasappel.html'
bar_pompl = 'arduino_local_csv/barcode_item_pomp_beker.html'

time_0 = time.time()

def pump_add(f):
    fin = open(f, "rt")
    data = fin.read()
    soup = BeautifulSoup(data, "html.parser")
    turns = soup.span
    turns_new = str(int(turns.get_text()) + 1)
    data = data.replace(turns.get_text(), turns_new)
    fin.close()
    fin = open(f, "wt")
    fin.write(data)
    fin.close()
    
def pump_update(f):
    fin = open("status.html", "rt")
    data = fin.read()
    soup = BeautifulSoup(data, "html.parser")
    turns = soup.find("span", class_= f)
    turns_new = str(int(turns.get_text()) + 1)
    print(turns)
    data = data.replace(str(turns), '<span class="s1_runs">' + turns_new + '</span>')
    print(turns)
    fin.close()
    fin = open("status.html", "wt")
    fin.write(data)
    fin.close()
    
def level_ok(f):
    fin = open("status.html", "rt")
    data = fin.read()
    data = data.replace('<span style="color: green;" class="' + f + '_level">OK</span>', '<span style="color: red;" class="' + f + '_level">LOW</span>')
    fin.close()
    fin = open("status.html", "wt")
    fin.write(data)
    
def level_low(f):
    fin = open("status.html", "rt")
    data = fin.read()
    data = data.replace('<span style="color: green;" class="' + f + '_level">OK</span>', '<span style="color: red;" class="' + f + '_level">LOW</span>')
    fin.close()
    fin = open("status.html", "wt")
    fin.write(data)
    fin.close()

def level_high(f):
    fin = open("status.html", "rt")
    data = fin.read()
    data = data.replace('<span style="color: green;" class="' + f + '_level">OK</span>', '<span style="color: red;" class="' + f + '_level">HIGH</span>')
    fin.close()
    fin = open("status.html", "wt")
    fin.write(data)
    fin.close()
    
def pump_on(f):
    data = fin.read()
    data = data.replace('<span style="color: red;" class="' + f + '_pomp">OFF</span>', '<span style="color: green;" class="' + f + '_pomp">ON</span>')
    fin.close()
    fin = open("status.html" "wt")
    fin.write(data)
    fin.close()
    
def pump_off(f):
    fin = open("status.html" "rt")
    data = fin.read()
    data = data.replace('<span style="color: green;" class="' + f + '_pomp">ON</span>', '<span style="color: red;" class="' + f + '_pomp">OFF</span>')
    fin.close()
    fin = open("status.html" "wt")
    fin.write(data)
    fin.close()
    
def add_order(f):
    current_time = str(time.asctime(time.localtime()))
    fin = open("status.html", "rt")
    data = fin.read()
    soup = BeautifulSoup(data, "html.parser")
    new_tag = soup.new_tag('li')
    new_tag.string = current_time + ": " + f
    soup.ol.insert(-1, new_tag)
    data = str(soup)
    fin.close()
    fin = open("status.html", "wt")
    fin.write(data)
    fin.close()
    
while True:
    current_time = round((time.time() - time_0) / 60, 2)
    ser_line = ser.readline()
    print(ser_line)
    try:
        ser_line = ser_line.decode("utf-8")
        if ser_line.find("smaak1_low") != -1:
            level_low('s1')
        elif ser_line.find("smaak1_ok") != -1:
            level_ok('s1')
        elif ser_line.find("smaak2_low") != -1:
            level_low('s2')
        elif ser_line.find("smaak2_ok") != -1:
            level_ok('s2')
        elif ser_line.find("smaak3_low") != -1:
            level_low('s3')
        elif ser_line.find("smaak3_ok") != -1:
            level_ok('s3')
        elif ser_line.find("water_low") != -1:
            level_low('w')
        elif ser_line.find("water_ok") != -1:
            level_ok('w')
        elif ser_line.find("limo_high") != -1:
            level_high('l')
        elif ser_line.find("limo_ok") != -1:
            level_ok('l')
        elif ser_line.find("turbiditysensor") != -1:
            print('a')
            turb_val = ser_line[ser_line.find(",")+1:ser_line.find(".")]
            with open("arduino.local/datalog_turb.csv","a") as f:
                writer = csv.writer(f,delimiter=",")
                writer.writerow([current_time,turb_val])
        elif ser_line.find("new_order") != -1:
            add_order(ser_line)
        elif ser_line.find("pompw_high") != -1:
            pump_add(bar_pompw)
            pump_update("w_runs")
            pump_on('w')
        elif ser_line.find("pompw_low") != -1:
            pump_off('w')
        elif ser_line.find("pompa_high") != -1:
            pump_add(bar_pompa)
            pump_update("a_runs")
            pump_on('a')
        elif ser_line.find("pompa_low") != -1:
            pump_off('a')
        elif ser_line.find("pompl_high") != -1:
            pump_add(bar_pompl)
            pump_update("l_runs")
            pump_on('l')
        elif ser_line.find("pompl_low") != -1:
            pump_off('l')
        elif ser_line.find("pomps1_high") != -1:
            pump_add(bar_pomps1)
            pump_update("s1_runs")
            pump_on('s1')
        elif ser_line.find("pomps1_low") != -1:
            pump_off('s1')
        elif ser_line.find("pomps2_high") != -1:
            pump_add(bar_pomps2)
            pump_update("s2_runs")
            pump_on('s1')
        elif ser_line.find("pomps2_low") != -1:
            pump_off('s2')
        elif ser_line.find("pomps3_high") != -1:
            pump_add(bar_pomps3)
            pump_update("s3_runs")
            pump_on('s2')
        elif ser_line.find("pomps3_low") != -1:
            pump_off('s3')
        else:
            print(ser_line)
    except:
        break

working_path = '/Users/kommunizm/Documents/Work/Uni/Technolab/clean_code/lemonadetechnolab.github.io'

def exit_handler():
    try:
     output = subprocess.check_output(['./git_commit.sh'], cwd=working_path, stderr=subprocess.STDOUT)
    except subprocess.CalledProcessError as e:
         print("Exception on process, rc=", e.returncode, "output=", e.output)

atexit.register(exit_handler)

SerialException: [Errno 2] could not open port /dev/cu.usbmodem14301: [Errno 2] No such file or directory: '/dev/cu.usbmodem14301'